In [1]:
import numpy as np
import pandas as pd
from PyPDF2 import PdfFileReader

# Load the PDF File

In [2]:
# open the PDF file
pdfFile = open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb')


# Read the File

In [3]:
# create PDFFileReader object to read the file
pdfReader = PdfFileReader(pdfFile)

# Check For Encryption

In [4]:
if pdfReader.isEncrypted == True:
    print("PDF File is encrypted. Password is needed to continue.")
else:
    print("PDF File is not encrypted. Password is not needed.")

PDF File is encrypted. Password is needed to continue.


# Print Document Info

In [5]:
#print("PDF File name: " + str(pdfReader.getDocumentInfo().title))
#print("PDF File created by: " + str(pdfReader.getDocumentInfo().creator))
#print("- - - - - - - - - - - - - - - - - - - -")

# Try Using PDFminer3

In [6]:
#imports
import pandas as pd

from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

In [77]:
#Set up resource manager to handle pdf content. text, images, etc.
resource_manager = PDFResourceManager()

#Used to display text
fake_file_handle = io.StringIO()

#Set up converter
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())

#Set up page interpreter
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/Users/johnathonsmith/Downloads/293RDCV.PEND.CASE_1642601189564_-1416322429.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  maxpages = 1,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)

                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022

                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1

        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #

PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY

        7815-OTH           02/22/1985 1                                                                  

        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            
        MENDEZ, OLGA LIDIA                                                                                         
        -------------------------------------------------------------------------------------------------------------
        90-11-10256-CV     11/08/1990 1                                                                  

        ATTORNEY GENERAL            

In [78]:
#Now try finding places to break it down a bit
print(repr(text))

'                COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GEN

In [79]:
#First, strip leading and trailing whitespaces
text = text.strip()
print(repr(text))

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL            

In [80]:
#Now separate header and body text info

#For header
header = text[:392]
print(repr(header))

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        '


In [81]:
#For body
text = text[392:]
print(repr(text))

'7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        -------------------------------------------------------------------------------------------------------------\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        -------------------------------------------------------------------------------------------------------------\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE OTHER T                     

In [82]:
#Now split each case by the dashed lines
cases = text.split('-------------------------------------------------------------------------------------------------------------')
cases

['7815-OTH           02/22/1985 1                                                                  \n\n        ATTORNEY GENERAL              MARIA LARSON              MENDEZ, CARLOS GILBERTO                            \n        MENDEZ, OLGA LIDIA                                                                                         \n        ',
 '\n        90-11-10256-CV     11/08/1990 1                                                                  \n\n        ATTORNEY GENERAL              ROLANDO MENCHACA          MARTINEZ, HECTOR RENE                              \n        MARTINEZ, ROSALINDA                                                                                        \n        ',
 '\n        90-12-10326-CV     12/14/1990 INJURY OR DAMAGE OTHER T                                           \n\n        DIAZ CARPENTER, ENRIQUETA     JAY T. KIMBROUGH          BOWLES JR., RUDY              RICHARD TINSMAN      \n        ',
 '\n        91-06-10598-CV     10/22/1993 MOTION ENFO

In [88]:
#Only work with one case for now
test_case = cases[-1]
print(repr(test_case))

'\n        97-06-14568-CV     06/16/1997 OTHER CIVIL CASES         02/19/1998  WON JURY TRIAL               \n\n        NUTMEG INSURANCE COMPANY      LOREN R. SMITH            MANCHA, GUILLERMO             MICHAEL J. BAGLEY    \n                                                                MARTINEZ, JOHNNY E.                                \n                                                                MAVERICK COUNTY'


In [90]:
#Strip the leading white space
test_case = test_case.lstrip()
print(repr(test_case))

'97-06-14568-CV     06/16/1997 OTHER CIVIL CASES         02/19/1998  WON JURY TRIAL               \n\n        NUTMEG INSURANCE COMPANY      LOREN R. SMITH            MANCHA, GUILLERMO             MICHAEL J. BAGLEY    \n                                                                MARTINEZ, JOHNNY E.                                \n                                                                MAVERICK COUNTY'


In [91]:
#Try splitting on the new lines
case_info = test_case.split('\n')

In [92]:
#I can isolate the cause number, file date, cause of action, docket date, docket type, 
#ans filed date, and cr number from here
case_info[0]

'97-06-14568-CV     06/16/1997 OTHER CIVIL CASES         02/19/1998  WON JURY TRIAL               '

In [94]:
#Date starts 19 characters in.
case_info[0][19:]

'06/16/1997 OTHER CIVIL CASES         02/19/1998  WON JURY TRIAL               '

In [95]:
#Cause of action start 30 characters in
case_info[0][30:]

'OTHER CIVIL CASES         02/19/1998  WON JURY TRIAL               '

In [99]:
#Docket date starts 56 characters in
case_info[0][56:]

'02/19/1998  WON JURY TRIAL               '

In [101]:
#Docket type starts 68 characters in
case_info[0][68:]

'WON JURY TRIAL               '

In [ ]:
#ANS Filed date starts 
#Still gotta find this one

In [ ]:
#CR number starts 
#Still gotta find this one

In [102]:
#Appears to be an extra new line
case_info[1]

''

In [103]:
#The first line of names starts 8 characters in. This should be the same for each case
case_info[2][8:]

'NUTMEG INSURANCE COMPANY      LOREN R. SMITH            MANCHA, GUILLERMO             MICHAEL J. BAGLEY    '

__Name Spacing__

The names appear to all start at the same place every time. Here is what I have found:

* Plaintiff Name: 8 Characters in
* Plaintiff Attorney: 38 Characters in
* Defendant Name: 64 Characters in
* Defendant Attorney: 94 Characters in

These numbers seem consistent regardless of what line they appear on.

In [104]:
#The second name starts 38 characters in. I hope this is the same for all of them
case_info[2][38:]

'LOREN R. SMITH            MANCHA, GUILLERMO             MICHAEL J. BAGLEY    '

In [105]:
#The third name starts 64 characters in. I hope this is also the same for all of them
case_info[2][64:]

'MANCHA, GUILLERMO             MICHAEL J. BAGLEY    '

In [110]:
#The fourth name starts 94 characters in
case_info[2][94:]

'MICHAEL J. BAGLEY    '

In [111]:
case_info[3]

'                                                                MARTINEZ, JOHNNY E.                                '

In [36]:
#The second plaintiff name starts 8 characters in. This should be the same for each case
case_info[3][8:]

'MENDEZ, OLGA LIDIA                                                                                         '

In [112]:
#The second defendant name starts 64 characters in.
case_info[3][64:]

'MARTINEZ, JOHNNY E.                                '

# Start Isolating Info

Isolate the case info and add it to a temp dict. When complete the temp dict will be appended to the container dictionary. Finally, when the info for each case has been extracted, convert the container dict to a dataframe.

In [117]:
#Create a container dict to store all case info
container_dict = {}

#Create a temp dict that stores individual case info
case_dict = {}

__County__

The county name will be found in the header info of each page. The easiest way I can think to isolate the county is to just use an if statement and check for each county name in the header text. There should only be one match, so it should be fine. Possible counties: Maverick, Dimmit, Zavala

In [115]:
header

'COUNTY OF MAVERICK 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/19/2022\n\n                                           RAN ON JANUARY 19, 2022 09:02am PAGE: 1\n\n        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #\n\nPLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY\n\n        '

In [116]:
#Use if statement to check for county names inside the header info
if header.count('MAVERICK') >= 1:
    county = 'Maverick'
elif header.count('DIMMIT') >= 1:
    county = 'Dimmit'
elif header.count('ZAVALA') >= 1:
    county = 'Zavala'
else:
    county = 'Something went wrong!'
    
county

'Maverick'

In [118]:
#Add to dict
case_dict['County'] = county

__Cause Number__

In [120]:
#Get the cause number
cause_num = case_info[0][:19].strip()
cause_num

'97-06-14568-CV'

In [121]:
#Add to dict
case_dict['Cause Number'] = cause_num

__File Date__

In [122]:
file_date = case_info[0][15:30].strip()
file_date

'06/16/1997'

In [123]:
#Add to dict
case_dict['File Date'] = file_date

__Cause of Action__

In [125]:
coa = case_info[0][30:56].strip()
coa

'OTHER CIVIL CASES'

In [126]:
#Add to dict
case_dict['Cause of Action'] = coa

__Docket Date__

In [128]:
docket_date = case_info[0][56:68].strip()
docket_date

'02/19/1998'

In [129]:
#Add to dict
case_dict['Docket Date'] = docket_date

__Docket Type__

In [131]:
docket_type = case_info[0][68:].strip()
docket_type

'WON JURY TRIAL'

In [132]:
#Add to dict
case_dict['Docket Type'] = docket_type

__ANS File Date__

I still need to figure this one out.

__CR Number__

I still need to figure this one out.

__Names__

The next few entries will be tricky. If there is more than one name associated with each label (plaintiff name, plaintiff attorney, defendant name, defendant attorney), then there will be more than one line of names. In my function, I'll have to set up a loop that can accurately pick up all of the names. For now, I'll make each label a list of names. If it needs to be changed to a string of names later, I can do that too. I'll also have to build a function that checks to see if each segment is composed entirely of whitespace. That way, if it is, we know there isn't a name for that particular label. 

In [159]:
#Will need to set up the lists before the loop. They will be added to the dict after the loop
plaintiff_names = []
plaintiff_attorneys = []
defendant_names = []
defendant_attorneys = []

#The names always start on the third line (index = 2)
for i in range(2, len(case_info)):
    #Get the plaintiff name on current line
    plaintiff_name = case_info[i][:38]
    
    #Check if plaintiff_name is all whitesapace. If not, strip it and add to names list
    #Also check that the string is not empty
    if plaintiff_name.isspace() == False and len(plaintiff_name) > 0:
        plaintiff_names.append(plaintiff_name.strip())
        
    #Get the plaintiff attorney on current line
    plaintiff_attorney = case_info[i][38:64]
    
    #Check if plaintiff_attorney is all whitespace. If not, strip it and add to list
    #Also check that the string is not empty
    if plaintiff_attorney.isspace() == False and len(plaintiff_attorney) > 0:
        plaintiff_attorneys.append(plaintiff_attorney.strip())
        
    #Get the defendant name on current line
    defendant_name = case_info[i][64:94]
    
    #Check if defendant_name is all whitespace. If not, strip it and add to list
    #Also check that the string is not empty
    if defendant_name.isspace() == False and len(defendant_name) > 0:
        defendant_names.append(defendant_name.strip())
    
    #Get the defendant attorney on current line
    defendant_attorney = case_info[i][94:]
    
    #Check if the defendant_attorney is all whitespace. If not, strip it and add to list
    #Also check that the string is not empty
    if defendant_attorney.isspace() == False and len(defendant_attorney) > 0:
        defendant_attorneys.append(defendant_attorney.strip())

In [160]:
#Now add all names to the dict
case_dict['Plaintiff Name'] = plaintiff_names
case_dict['Plaintiff Attorney'] = plaintiff_attorneys
case_dict['Defendant Name'] = defendant_names
case_dict['Defendant Attorney'] = defendant_attorneys

In [162]:
case_dict

{'County': 'Maverick',
 'Cause Number': '97-06-14568-CV',
 'File Date': '06/16/1997',
 'Cause of Action': 'OTHER CIVIL CASES',
 'Docket Date': '02/19/1998',
 'Docket Type': 'WON JURY TRIAL',
 'Plaintiff Name': ['NUTMEG INSURANCE COMPANY'],
 'Plaintiff Attorney': ['LOREN R. SMITH'],
 'Defendant Name': ['MANCHA, GUILLERMO',
  'MARTINEZ, JOHNNY E.',
  'MAVERICK COUNTY'],
 'Defendant Attorney': ['MICHAEL J. BAGLEY']}